In [96]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np

class ConvNetwork(nn.Module):
    
    def __init__(self):
        super(ConvNetwork,self).__init__()
        #[in_channels,out_channels,kernel,stride,padding]
        layer1=[3,16,3,1,1]
        layer2=[16,32,3,1,1]
        layer3=[32,64,3,1,1]
        self.conv1=torch.nn.Conv2d(layer1[0],layer1[1],kernel_size=layer1[2],stride=layer1[3],padding=layer1[4])
        self.conv2=torch.nn.Conv2d(layer2[0],layer2[1],kernel_size=layer2[2],stride=layer2[3],padding=layer2[4])
        self.conv3=torch.nn.Conv2d(layer3[0],layer3[1],kernel_size=layer3[2],stride=layer3[3],padding=layer3[4])
        self.fc1=nn.Linear(64*32*32,100)
        self.fc2=torch.nn.Linear(100,50)
        self.fc3=torch.nn.Linear(50,4)
    
        
    def forward(self,inp):
        out1=F.relu(self.conv1(inp))
        out2=F.relu(self.conv2(out1))
        out3=F.relu(self.conv3(out2))
        out=out3.view(-1,out3.shape[1]*out3.shape[2]*out3.shape[3])
        outfc1=self.fc1(out)
        outfc2=self.fc2(outfc1)
        outfc3=self.fc3(outfc2)
        outf=F.softmax(outfc3,_stacklevel=4)
        return outf

model = ConvNetwork()

In [92]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def data2():
    traind=[]
    trainl=[]
    testd=[]
    testl=[]
    keys={1:0,3:1,5:2,9:3}
    for i in range(5):
        d=unpickle('./data/cifar-10-batches-py/data_batch_'+str(i+1))
        data=d[b'data']
        lbl=d[b'labels']
        for i in range(len(data)):
            if(lbl[i] in keys):
                img=np.reshape(data[i],(32,32,3))
                nimg=[]
                i1=img[:,:,0]
                i2=img[:,:,1]
                i3=img[:,:,2]
                nimg.append(i1)
                nimg.append(i2)
                nimg.append(i3)
                traind.append(nimg)
                trainl.append(keys[lbl[i]])
    d2=unpickle('./data/cifar-10-batches-py/test_batch')
    data2=d2[b'data']
    lbl2=d2[b'labels']
    for i in range(len(data2)):
        if(lbl2[i] in keys):
            img=np.reshape(data2[i],(32,32,3))
            nimg=[]
            i1=img[:,:,0]
            i2=img[:,:,1]
            i3=img[:,:,2]
            nimg.append(i1)
            nimg.append(i2)
            nimg.append(i3)
            testd.append(nimg)
            testl.append(keys[lbl2[i]])
        
#     for i in range(len(data2)):
        
#         img=cv2.cvtColor(np.reshape(data2[i],(32,32,3)),cv2.COLOR_BGR2GRAY)
#         img=np.ravel(img)
#         testd.append(img)
#         testl.append(lbl2[i])
    a=np.asarray(traind)
    b=np.asarray(trainl)
    c=np.asarray(testd)
    d=np.asarray(testl)
    return a,b,c,d

In [93]:
m=[0.485, 0.456, 0.406]
s=[0.229, 0.224, 0.225]
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=m, std=s)])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [94]:
test=[]
for x,y in (trainset):
    if(y==1):
        test.append((x,y))
    elif(y==3):
        test.append((x,y))
    elif(y==5):
        test.append((x,y))
    elif(y==9):
        test.append((x,y))

train=[]
for x,y in (trainset):
    if(y==1):
        train.append((x,y))
    elif(y==3):
        train.append((x,y))
    elif(y==5):
        train.append((x,y))
    elif(y==9):
        train.append((x,y))

# print(np.shape(train),np.shape(test))

In [97]:
#[batchsize,learning_rate,epochs,classes,momentum,worker]
p=[100,0.001,3,4,0.9,2]

trainloader = torch.utils.data.DataLoader(train, batch_size=p[0],shuffle=True, num_workers=p[5])
testloader = torch.utils.data.DataLoader(test, batch_size=p[0],shuffle=False, num_workers=p[5])

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=p[1],momentum=p[4])


for i in range(p[2]):
    curr=0
    l=0.0
    for j,data in enumerate(trainloader,0):
        x,y=data
        y[y==1] = torch.from_numpy(np.zeros((y[y==1].size())).astype(np.long))
        y[y==3] = torch.from_numpy(np.zeros((y[y==3].size())).astype(np.long)+1)
        y[y==5] = torch.from_numpy(np.zeros((y[y==5].size())).astype(np.long)+2)
        y[y==9] = torch.from_numpy(np.zeros((y[y==9].size())).astype(np.long)+3)
        
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        l += loss.item()
        if i % 100 == 99:
          print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
          l = 0.0
        
#     for j in range(batches):
#         trnx=[]
#         trny=[]
#         for k in range(bsize):
#             trnx.append(train_x[k+curr])
#             trny.append(train_y[k+curr])
#         curr=curr+bsize
#         trnx=np.array(trnx).astype(np.float32)
#         trny=np.array(trny).astype(np.long)
#         trnx=torch.from_numpy(trnx)
#         trny=torch.from_numpy(trny)

#         outputs = model(trnx)
#         optimizer.zero_grad()
#         loss = criterion(outputs, trny)
#         loss.backward()
#         optimizer.step()
#         if(j%100==0):
#             print(j,i,loss.item())


In [ ]:
model.eval()
with torch.no_grad():
    for data in test loader:
        
for i in range(p[2]):
    curr=0
    l=0.0
    for j,data in enumerate(trainloader,0):
        x,y=data
        y[y==1] = torch.from_numpy(np.zeros((y[y==1].size())).astype(np.long))
        y[y==3] = torch.from_numpy(np.zeros((y[y==3].size())).astype(np.long)+1)
        y[y==5] = torch.from_numpy(np.zeros((y[y==5].size())).astype(np.long)+2)
        y[y==9] = torch.from_numpy(np.zeros((y[y==9].size())).astype(np.long)+3)
        
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        l += loss.item()


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
model.eval()
with torch.no_grad():
    accurate=0
    total=0
    y_pred=[]
    curr=0
    for i in range(200):
        tstx=[]
        tsty=[]
        for j in range(bsize):
            tstx.append(test_x[j+curr])
            tsty.append(test_y[j+curr])
        curr=curr+bsize
        tstx=np.array(tstx).astype(np.float32)
        tsty=np.array(tsty).astype(np.long)
        
        print(np.shape(tstx))

        tstx=torch.from_numpy(tstx)
        tsty=torch.from_numpy(tsty)
        
        output=model(tstx)
        _, predicted=torch.max(output.data,1)
        y_pred.extend(predicted.numpy().tolist())
        total=total+tsty.size(0)
        accurate=accurate+(predicted==tsty).sum().item()
    print(np.shape(y_pred))
#     y_pred=clf.predict(test_x)
    cm = confusion_matrix(test_y, y_pred)
    print(np.sum(np.diagonal(cm))/np.sum(cm))
    plt.clf()
    plt.matshow(cm)
    plt.colorbar()
    plt.title("Confusion Matrix")
    plt.savefig("ConfusionLR")
    print("Accuracy :",accurate/total)


train_x=torch.from_numpy(train_x)
train_y=torch.from_numpy(train_y)
test_x=torch.from_numpy(test_x)
test_y=torch.from_numpy(test_y)
print(train_x.size(),train_y.size(),test_x.size(),test_y.size())

In [18]:
import tensorflow as tf

#[batchsize,learning_rate,epochs,classes]
param = [100,0.001,5,4]
trainset = torch.utils.data.DataLoader(dataset=traindata,batch_size=param[0],shuffle=True)
testset = torch.utils.data.DataLoader(dataset=testdata,batch_size=param[0],shuffle=False)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=param[1])


# Train the model
total_step = len(trainset)
for epoch in range(param[2]):
    for i, data in enumerate(trainset):
        images, labels = data
        imgs1=images[labels==1]
        imgs3=images[labels==3]
        imgs5=images[labels==5]
        imgs9=images[labels==9]
        imgs1=torch.cat([imgs1, imgs3], 0)
        imgs1=torch.cat([imgs1, imgs5], 0)
        imgs1=torch.cat([imgs1, imgs9], 0)
#         tf.concat([imgs1, imgs5], 0)
#         tf.concat([imgs1, imgs9], 0)
        
        lbl1=labels[labels==1]
        lbl3=labels[labels==3]
        lbl5=labels[labels==5]
        lbl9=labels[labels==9]
        lbl1=torch.cat([lbl1, lbl3], 0)
        lbl1=torch.cat([lbl1, lbl5], 0)
        lbl1=torch.cat([lbl1, lbl9], 0)
#         tf.concat([lbl1, lbl3], 0)
#         tf.concat([lbl1, lbl5], 0)
#         tf.concat([lbl1, lbl9], 0)
    
        outputs = model(imgs1)
        print(np.shape(outputs),np.shape(lbl1))
        loss = criterion(outputs, lbl1)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, param[2], i+1, total_step, loss.item()))

Epoch [1/5], Step [100/500], Loss: 2.2286
Epoch [1/5], Step [200/500], Loss: 2.1884
Epoch [1/5], Step [300/500], Loss: 2.1278
Epoch [1/5], Step [400/500], Loss: 2.0862
Epoch [1/5], Step [500/500], Loss: 2.2566
Epoch [2/5], Step [100/500], Loss: 2.2720
Epoch [2/5], Step [200/500], Loss: 2.2945
Epoch [2/5], Step [300/500], Loss: 2.2667
Epoch [2/5], Step [400/500], Loss: 2.2271
Epoch [2/5], Step [500/500], Loss: 2.2667
Epoch [3/5], Step [100/500], Loss: 2.1516
Epoch [3/5], Step [200/500], Loss: 2.1612
Epoch [3/5], Step [300/500], Loss: 2.1846
Epoch [3/5], Step [400/500], Loss: 2.1980
Epoch [3/5], Step [500/500], Loss: 2.2660
Epoch [4/5], Step [100/500], Loss: 2.2612
Epoch [4/5], Step [200/500], Loss: 2.2220
Epoch [4/5], Step [300/500], Loss: 2.2469
Epoch [4/5], Step [400/500], Loss: 2.2612
Epoch [4/5], Step [500/500], Loss: 2.2945
Epoch [5/5], Step [100/500], Loss: 2.2339
Epoch [5/5], Step [200/500], Loss: 2.3260
Epoch [5/5], Step [300/500], Loss: 2.2751
Epoch [5/5], Step [400/500], Loss:

In [19]:
model.eval()
with torch.no_grad():
    accurate=0
    total=0
    for images, labels in testset:
        imgs1=images[labels==1]
        imgs3=images[labels==3]
        imgs5=images[labels==5]
        imgs9=images[labels==9]
        imgs1=torch.cat([imgs1, imgs3], 0)
        imgs1=torch.cat([imgs1, imgs5], 0)
        imgs1=torch.cat([imgs1, imgs9], 0)
#         tf.concat([imgs1, imgs3], 0)
#         tf.concat([imgs1, imgs5], 0)
#         tf.concat([imgs1, imgs9], 0)
        
#         lbl1=labels[labels==1]
#         lbl3=labels[labels==3]
#         lbl5=labels[labels==5]
#         lbl9=labels[labels==9]
#         lbl1=torch.cat([lbl1, lbl3], 0)
#         lbl1=torch.cat([lbl1, lbl5], 0)
#         lbl1=torch.cat([lbl1, lbl9], 0)
        
        
        lbl1=np.zeros(len(labels[labels==1]))
        lbl3=np.zeros(len(labels[labels==3]))+1
        lbl5=np.zeros(len(labels[labels==5]))+2
        lbl9=np.zeros(len(labels[labels==9]))+3
        lbl1=np.concatenate((lbl1, lbl3), axis=0)
        lbl1=np.concatenate((lbl1, lbl5), axis=0)
        lbl1=np.concatenate((lbl1, lbl9), axis=0)
        
        lbl1 = torch.tensor(lbl1.astype(np.long))
        
        
#         tf.concat([lbl1, lbl3], 0)
#         tf.concat([lbl1, lbl5], 0)
#         tf.concat([lbl1, lbl9], 0)
        output=model(imgs1)
        _, predicted=torch.max(output.data,1)
        p=predicted.numpy()
        total=total+lbl1.size(0)
        accurate=accurate+(predicted==lbl1).sum().item()
#         print(accurate)
    print("Accuracy :",accurate/total)

torch.save(model.state_dict(),"Model.ckpt")

Accuracy : 0.25
